In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-11 06:43:49.797024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 06:43:50.842944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 06:43:50.843064: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 06:43:50.843073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT li

In [1]:
order = [
    155, 153,151,149,147,144,142,140, 138, 143,141,139, 137,135,133,130,131,
129,127,125,123,121,119,117,115,113,111,109,107,105,103, 101, 99, 97, 95, 93,
91,89,87,85,81,83,79,77,75,73,71,67,65,63,61,59,57,55,53,51,49,47,45,43,41,39,
37,33,31,29,27,25,28,30,32,34,36,38,40,42,44,48,50,52,54,56,58,60,62,
    64,66,68,72,74,76,78,80,82,84,88,90,92,94,96,98,100,102,104,106,108,
    110,112,114,116,118,120,122,124,136,134,132,128,126
]
data_folder = "/dm7100f/yifan/StereoSeq_macaque_processed"
files = [f'macaque_T{o}.h5ad' for o in order]

In [2]:
# data_folder = "./results/split_data/"
results_folder = "./results/PASTE2/"
figures_folder = "./results/figures/PASTE2"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(figures_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
## Perform the Spateo alignment
from tqdm import tqdm
import time
sampling_num = 5000
spatial_key = 'r_spatial'
key_added = 'align_spatial'
for i in tqdm(range(len(files)-1)):
    slice1 = ad.read_h5ad(os.path.join(data_folder, files[i]))
    slice2 = ad.read_h5ad(os.path.join(data_folder, files[i+1]))
    slice1.obsm[spatial_key] = slice1.obsm[spatial_key].astype(np.float64)
    slice2.obsm[spatial_key] = slice2.obsm[spatial_key].astype(np.float64)
    slice1 = slice1[~np.isnan(slice1.obsm[spatial_key].sum(1))]
    slice2 = slice2[~np.isnan(slice2.obsm[spatial_key].sum(1))]
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    top_genes_num = 5000
    top_genes_1 = np.argsort(-slice1.X.sum(0)).A[0][:top_genes_num] if slice1.shape[1] > top_genes_num else np.arange(slice1.shape[1])
    top_genes_2 = np.argsort(-slice2.X.sum(0)).A[0][:top_genes_num] if slice2.shape[1] > top_genes_num else np.arange(slice2.shape[1])
    slice1 = slice1[:,top_genes_1]
    slice2 = slice2[:,top_genes_2]
    
    time_start = time.time()
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
        fixed_s=0.99,
    )
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                     | 0/118 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.020867e+00|0.000000e+00|0.000000e+00
    1|2.745212e+00|1.004129e-01|2.756547e-01
    2|2.744487e+00|2.642041e-04|7.251049e-04
    3|2.739466e+00|1.832946e-03|5.021293e-03
    4|2.725463e+00|5.137849e-03|1.400302e-02
    5|2.712645e+00|4.725165e-03|1.281770e-02
    6|2.706863e+00|2.136301e-03|5.782672e-03
    7|2.704696e+00|8.011592e-04|2.166892e-03
    8|2.703671e+00|3.788429e-04|1.024267e-03
    9|2.703153e+00|1.918727e-04|5.186613e-04
   10|2.702885e+00|9.894677e-05|2.674418e-04
   11|2.702729e+00|5.803708e-05|1.568585e-04
   12|2.702617e+00|4.118947e-05|1.113194e-04
   13|2.702557e+00|2.230262e-05|6.027410e-05
   14|2.702530e+00|9.942118e-06|2.686887e-05
   15|2.702515e+00|5.664974e-06|1.530968e-05
   16|2.702501e+00|4.937276e-06|1.334300e-05
   17|2.702494e+00|2.685845e-06|7.258481e-06
   18|2.702490e+00|1.661866e-06|4.491175e-06
   19|2.702485e+00|1.738285e-06|

  1%|         | 1/118 [07:19<14:16:57, 439.46s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.270246e+00|0.000000e+00|0.000000e+00
    1|2.904073e+00|1.260895e-01|3.661730e-01
    2|2.891417e+00|4.377209e-03|1.265633e-02
    3|2.854715e+00|1.285638e-02|3.670131e-02
    4|2.846447e+00|2.904600e-03|8.267792e-03
    5|2.844176e+00|7.987531e-04|2.271794e-03
    6|2.843655e+00|1.831722e-04|5.208786e-04
    7|2.843562e+00|3.249913e-05|9.241330e-05
    8|2.843551e+00|4.087446e-06|1.162286e-05
    9|2.843550e+00|4.171680e-07|1.186238e-06
 1000|2.843550e+00|1.561742e-16|4.440892e-16


  2%|▏         | 2/118 [10:48<9:47:08, 303.70s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


...

PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.017627e+00|0.000000e+00|0.000000e+00
    1|2.688603e+00|1.223775e-01|3.290244e-01
    2|2.675229e+00|4.999212e-03|1.337403e-02
    3|2.653552e+00|8.168850e-03|2.167647e-02
    4|2.644616e+00|3.378803e-03|8.935637e-03
    5|2.643639e+00|3.698947e-04|9.778678e-04
    6|2.643601e+00|1.409826e-05|3.727018e-05
    7|2.643598e+00|1.281885e-06|3.388790e-06
    8|2.643597e+00|2.737546e-07|7.236968e-07
    9|2.643597e+00|9.899563e-08|2.617045e-07
 1000|2.643597e+00|0.000000e+00|0.000000e+00


  7%|▋         | 8/118 [31:49<6:15:28, 204.81s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.267368e+00|0.000000e+00|0.000000e+00
    1|2.854269e+00|1.447301e-01|4.130986e-01
    2|2.844515e+00|3.428857e-03|9.753436e-03
    3|2.828288e+00|5.737704e-03|1.622788e-02
    4|2.822244e+00|2.141392e-03|6.043531e-03
    5|2.814269e+00|2.833821e-03|7.975135e-03
    6|2.811334e+00|1.044053e-03|2.935182e-03
    7|2.810876e+00|1.627668e-04|4.575173e-04
    8|2.810717e+00|5.653382e-05|1.589006e-04
    9|2.810636e+00|2.876446e-05|8.084643e-05
   10|2.810565e+00|2.535499e-05|7.126186e-05
   11|2.810522e+00|1.537797e-05|4.322013e-05
   12|2.810491e+00|1.104605e-05|3.104481e-05
   13|2.810471e+00|7.234580e-06|2.033257e-05
   14|2.810460e+00|3.692276e-06|1.037699e-05
   15|2.810452e+00|2.943378e-06|8.272222e-06
   16|2.810446e+00|2.025494e-06|5.692542e-06
   17|2.810443e+00|1.235047e-06|3.471029e-06
   18|2.810441e+00|4.925601e-07|1.384311e-06
   19|2.810441e+00|2.415529e-07|

  8%|▊         | 9/118 [38:21<7:58:41, 263.50s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/Macaque/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.000212e+00|0.000000e+00|0.000000e+00
    1|2.702359e+00|1.102195e-01|2.978527e-01
    2|2.681287e+00|7.859081e-03|2.107245e-02
    3|2.654207e+00|1.020263e-02|2.707988e-02
    4|2.647569e+00|2.507121e-03|6.637778e-03


In [ ]:
## Perform the Moscot alignment
sampling_num = 5000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
        fixed_s=0.99,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1), np.sum(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                                                                          | 0/18 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/OpenST/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.502832e+00|0.000000e+00|0.000000e+00
    1|1.329600e+00|1.302889e-01|1.732321e-01
    2|1.326029e+00|2.693032e-03|3.571038e-03
    3|1.325476e+00|4.170764e-04|5.528246e-04
    4|1.325319e+00|1.186692e-04|1.572745e-04
    5|1.325292e+00|2.027759e-05|2.687371e-05
    6|1.325286e+00|4.572811e-06|6.060281e-06
    7|1.325285e+00|6.798674e-07|9.010179e-07
    8|1.325285e+00|6.175610e-08|8.184441e-08
    9|1.325284e+00|1.912368e-07|2.534432e-07
   10|1.325284e+00|9.912220e-08|1.313651e-07
 1000|1.325284e+00|0.000000e+00|0.000000e+00


  6%|██████████▋                                                                                                                                                                                      | 1/18 [03:30<59:37, 210.43s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/OpenST/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.560748e+00|0.000000e+00|0.000000e+00
    1|1.384350e+00|1.274228e-01|1.763977e-01
    2|1.380164e+00|3.033192e-03|4.186302e-03
    3|1.379055e+00|8.042168e-04|1.109059e-03
    4|1.378833e+00|1.608624e-04|2.218024e-04
    5|1.378806e+00|1.968093e-05|2.713617e-05
    6|1.378804e+00|1.216441e-06|1.677233e-06
    7|1.378803e+00|3.666284e-07|5.055085e-07
 1000|1.378803e+00|3.220831e-16|4.440892e-16


 11%|█████████████████████▍                                                                                                                                                                           | 2/18 [06:20<49:43, 186.45s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/OpenST/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.621255e+00|0.000000e+00|0.000000e+00
    1|1.442244e+00|1.241200e-01|1.790113e-01
    2|1.437899e+00|3.021576e-03|4.344721e-03
    3|1.437024e+00|6.085661e-04|8.745244e-04
    4|1.436945e+00|5.509150e-05|7.916348e-05
    5|1.436939e+00|4.386193e-06|6.302692e-06
    6|1.436938e+00|7.897646e-07|1.134843e-06
    7|1.436938e+00|7.249172e-08|1.041661e-07
 1000|1.436938e+00|3.090525e-16|4.440892e-16


 17%|████████████████████████████████▏                                                                                                                                                                | 3/18 [09:08<44:31, 178.10s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/OpenST/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|1.384162e+00|0.000000e+00|0.000000e+00
    1|1.226805e+00|1.282656e-01|1.573570e-01
    2|1.226102e+00|5.734487e-04|7.031068e-04
    3|1.225997e+00|8.624762e-05|1.057393e-04
    4|1.225915e+00|6.672532e-05|8.179956e-05
    5|1.225736e+00|1.458752e-04|1.788044e-04
    6|1.225428e+00|2.511290e-04|3.077406e-04
    7|1.225069e+00|2.930415e-04|3.589961e-04
    8|1.224853e+00|1.762484e-04|2.158785e-04
